### Setting Up

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/ECE1786_Project/

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1xRc8WnihQYZ-NqUh9-SHkHVkkk4-RtFd/ECE1786_Project


In [2]:
!pip install wandb datasets transformers sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 4.5 MB/s 
     |████████████████████████████████| 451 kB 85.7 MB/s 
     |████████████████████████████████| 5.8 MB 78.6 MB/s 
     |████████████████████████████████| 1.3 MB 80.3 MB/s 
     |████████████████████████████████| 182 kB 95.8 MB/s 
     |████████████████████████████████| 168 kB 93.9 MB/s 
     |████████████████████████████████| 62 kB 1.6 MB/s 
     |████████████████████████████████| 168 kB 93.3 MB/s 
     |████████████████████████████████| 166 kB 93.5 MB/s 
     |████████████████████████████████| 166 kB 90.0 MB/s 
     |████████████████████████████████| 162 kB 93.2 MB/s 
     |████████████████████████████████| 162 kB 88.7 MB/s 
     |████████████████████████████████| 158 kB 97.8 MB/s 
     |████████████████████████████████| 157 kB 98.4 MB/s 
     |████████████████████████████████| 157 kB 95.0 MB/s 
     |████████████████████████████

In [3]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

### Importing Data

In [4]:
import pandas as pd
Training = pd.read_csv('Training.csv')
Validation = pd.read_csv('Validation.csv')
Test = pd.read_csv('Test.csv')

In [5]:
Training.head()

,text,code
0,3 Write a program to find the element that app...,def bubbleSort(array):\n swapped = False\n ...
1,5 write a python program to find the third max...,def bubbleSort(array):\n swapped = False\n ...
2,6 Write a program to find the maximum product ...,def bubbleSort(array):\n swapped = False\n ...
3,7 write a python program to find the largest p...,def bubbleSort(array):\n swapped = False\n ...
4,8 write a Python program to print an array of ...,def bubbleSort(array):\n swapped = False\n ...


In [6]:
Validation.head()

,text,code
0,1 write a python program to sort an array in a...,def bubbleSort(array):\n swapped = False\n ...
1,12 write a python program to sort an array in ...,def selectionSort(array):\n for i in range(...
2,23 write a python program to find the element ...,def insertionSort(array):\n for i in range(...
3,34 write a python program to check if any valu...,import math\ndef shellSort(array):\n n = le...
4,45 write a python program to find the third di...,"def heapify(array, n, i):\n smallest = i\n ..."


In [7]:
Test.head()

,text,code
0,2 Write python code to sort an array in descen...,def bubbleSort(array):\n swapped = False\n ...
1,4 write a python program to check if any value...,def bubbleSort(array):\n swapped = False\n ...
2,13 Write python code to determine the element ...,def selectionSort(array):\n for i in range(...
3,15 Use the Selection Sort algorithm to find th...,def selectionSort(array):\n for i in range(...
4,24 Write python code to check if any value app...,def insertionSort(array):\n for i in range(...


In [8]:
from datasets import Dataset
training_set = Dataset.from_pandas(Training)
validation_set = Dataset.from_pandas(Validation)
test_set = Dataset.from_pandas(Test)

In [9]:
training_set

Dataset({
    features: ['text', 'code'],
    num_rows: 202
})

In [10]:
validation_set

Dataset({
    features: ['text', 'code'],
    num_rows: 28
})

In [11]:
test_set

Dataset({
    features: ['text', 'code'],
    num_rows: 101
})

### Model Definition

In [12]:
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained("Salesforce/codet5-small")

model = AutoModelWithLMHead.from_pretrained("Salesforce/codet5-small")

Downloading:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/703k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/294k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/transformers/models/auto/modeling_auto.py:1177: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


Downloading:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

### Pre-Processing

In [13]:
def prepare_data(examples):
    text = examples['text']
    code = examples['code']
    
    model_inputs = tokenizer(text, padding="longest", truncation = True)
    labels = tokenizer(code, padding="longest", truncation = True).input_ids

    labels_with_ignore_index = []
    for labels_example in labels:
      labels_example = [label if label != 0 else -100 for label in labels_example]
      labels_with_ignore_index.append(labels_example)
    
    model_inputs["labels"] = labels_with_ignore_index

    return model_inputs

In [14]:
tokenized_training_set = training_set.map(prepare_data, batched=True, remove_columns=training_set.column_names)
tokenized_validation_set = validation_set.map(prepare_data, batched=True, remove_columns=validation_set.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [15]:
labels = tokenized_training_set["labels"][0]
print(tokenizer.decode([label for label in labels if label != -100]))

<s>def bubbleSort(array):
    swapped = False
    for i in range(len(array)-1,0,-1):
        for j in range(i):
            if array[j]>array[j+1]:
                array[j], array[j+1] = array[j+1], array[j]
                swapped= True
        if swapped:
            swapped=False
        else:
            break
    return array
arr = [2,2,1,1,1,2,2]
arr = bubbleSort(arr)
print(arr[len(arr)//2])








</s>


### Training & Saving

In [16]:
from transformers import TrainingArguments

args = TrainingArguments(
    "/content/drive/MyDrive/ECE1786_Project/results",
    logging_steps = 51,
    evaluation_strategy = "epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=10,
    learning_rate=5e-5,
    weight_decay=0.01,
    report_to="wandb"
)

In [17]:
from transformers import default_data_collator

data_collator = default_data_collator

In [18]:
from transformers import Trainer

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_training_set,
    eval_dataset=tokenized_validation_set,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [19]:
import torch, gc
torch.cuda.empty_cache()
gc.collect()

28

In [20]:
# wandb.init(project="codet5_base_finetuning", entity="dhairya18498")

In [21]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 202
  Num Epochs = 10
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 510
  Number of trainable parameters = 60492288
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: blaredai1997. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,3.731000,1.955458
2,1.944700,1.341588
3,1.541900,1.075457
4,1.300400,0.886330
5,1.131900,0.766920
6,1.007400,0.680645
7,0.936600,0.618070
8,0.879900,0.584811
9,0.846900,0.554680
10,0.828200,0.546745


***** Running Evaluation *****
  Num examples = 28
  Batch size = 4
***** Running Evaluation *****
  Num examples = 28
  Batch size = 4
***** Running Evaluation *****
  Num examples = 28
  Batch size = 4
***** Running Evaluation *****
  Num examples = 28
  Batch size = 4
***** Running Evaluation *****
  Num examples = 28
  Batch size = 4
***** Running Evaluation *****
  Num examples = 28
  Batch size = 4
***** Running Evaluation *****
  Num examples = 28
  Batch size = 4
***** Running Evaluation *****
  Num examples = 28
  Batch size = 4
***** Running Evaluation *****
  Num examples = 28
  Batch size = 4
Saving model checkpoint to /content/drive/MyDrive/ECE1786_Project/results/checkpoint-500
Configuration saved in /content/drive/MyDrive/ECE1786_Project/results/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/ECE1786_Project/results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/ECE1786_Project/results/checkpoint-500/tokeni

TrainOutput(global_step=510, training_loss=1.41488603330126, metrics={'train_runtime': 78.7314, 'train_samples_per_second': 25.657, 'train_steps_per_second': 6.478, 'total_flos': 23494490849280.0, 'train_loss': 1.41488603330126, 'epoch': 10.0})

In [22]:
trainer.save_model("codet5_small_baseline")

Saving model checkpoint to codet5_small_baseline
Configuration saved in codet5_small_baseline/config.json
Model weights saved in codet5_small_baseline/pytorch_model.bin
tokenizer config file saved in codet5_small_baseline/tokenizer_config.json
Special tokens file saved in codet5_small_baseline/special_tokens_map.json


In [23]:
model = AutoModelWithLMHead.from_pretrained("codet5_small_baseline")

/usr/local/lib/python3.8/dist-packages/transformers/models/auto/modeling_auto.py:1177: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
loading configuration file codet5_small_baseline/config.json
Model config T5Config {
  "_name_or_path": "codet5_small_baseline",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "bos_token_id": 1,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 2,
  "feed_forward_proj": "relu",
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-06,
  "model_

### Evaluation

In [24]:
import random

In [25]:
sample_input = "write a python program for Recursive Insertion Sort"
print('sample_input:', sample_input, '\n')
input_ids = tokenizer(sample_input, return_tensors='pt').input_ids
outputs = model.generate(input_ids, max_new_tokens=500)
print('generated output:\n', tokenizer.decode(outputs[0], skip_special_tokens=True))

sample_input: write a python program for Recursive Insertion Sort 

generated output:
 def insertionSortRecursive(insertionSortRecursive):
    if insertionSortRecursive:
        insertionSortRecursive(insertionSortRecursive, reverse=True)
        insertionSortRecursive(insertionSortRecursive, reverse=False)
        insertionSortRecursive(insertionSortRecursive, reverse=True)
        insertionSortRecursive(insertionSortRecursive, reverse=True)
        insertionSortRecursive(insertionSortRecursive, reverse=False)
        insertionSortRecursive(insertionSortRecursive, reverse=True, reverse=False, reverse=True) -> list:
            if insertionSortRecursive(insertionSortRecursive, reverse=False, reverse=False, reverse=False)
        insertionSortRecursive(insertionSortRecursive, reverse=True)
        insertionSortRecursive(insertionSortRecursive, reverse=False)
        insertionSortRecursive(insertionSortRecursive, reverse=False)
        insertionSortRecursive(insertionSortRecursive, rever

In [28]:
def insertionSortRecursive(insertionSortRecursive):
    if insertionSortRecursive:
        insertionSortRecursive(insertionSortRecursive, reverse=True)
        insertionSortRecursive(insertionSortRecursive, reverse=False)
        insertionSortRecursive(insertionSortRecursive, reverse=True)
        insertionSortRecursive(insertionSortRecursive, reverse=True)
        insertionSortRecursive(insertionSortRecursive, reverse=False)
        insertionSortRecursive(insertionSortRecursive, reverse=True, reverse=False, reverse=True) -> list:
            if insertionSortRecursive(insertionSortRecursive, reverse=False, reverse=False, reverse=False)
        insertionSortRecursive(insertionSortRecursive, reverse=True)
        insertionSortRecursive(insertionSortRecursive, reverse=False)
        insertionSortRecursive(insertionSortRecursive, reverse=False)
        insertionSortRecursive(insertionSortRecursive, reverse=True, recursive=True, reverse=False, reverse=False)
        if not reverse:
        insertionSortRecursive(insertionSortRecursive(insertionSortRecursive, reverse=True)
        if not mergeRecursive(insertionSortRecursive(insertionSortRecursive(insertionSortRecursive)
        insertionSortRecursive(insertionSortRecursive(insertionSortRecursive, reverse=True)
        if not insertionSortRecursive(insertionSortRecursive, reverse=True, recursive=True, recursive=False, reverse=True)
        if not mergeRecursive(insertionSortRecursive(insertionSortRecursive)
        else:
        insertionSortRecursive(insertionSortRecursive(insertionSortRecursive(insertionSortRecursive)
        if not mergeRecursive)):
        insertionSortRecursive(insertionSortRecursive)

SyntaxError: ignored

In [26]:
sample_input = "using Cycle Sort, sort an input array and print the third maximum element"
print('sample_input:', sample_input, '\n')
input_ids = tokenizer(sample_input, return_tensors='pt').input_ids
outputs = model.generate(input_ids, max_new_tokens=500)
print('generated output:\n', tokenizer.decode(outputs[0], skip_special_tokens=True))

sample_input: using Cycle Sort, sort an input array and print the third maximum element 

generated output:
 def cycleSort(array):
    n = len(array)
    for i in range(n):
        if array[i] > array[i+1]:
            array[i], array[i+1] = array[i+1], array[i+1]
                array[i], array[i+1] = array[i+1], array[i+1]
                array[i] = array[i+1], array[i]
                array[i]
                array[i]
                array[i] = array[i+1]
                array[i+1]
                array[i+1]
                array[i+1]
                array[i]
                array[i]
                array[i+1]
                array[i+1]
                array[i]
                array[i+1]
                array[i+1]
                array[i+1]
                print(cycleSort(array))












































































































In [29]:
def cycleSort(array):
    n = len(array)
    for i in range(n):
        if array[i] > array[i+1]:
            array[i], array[i+1] = array[i+1], array[i+1]
                array[i], array[i+1] = array[i+1], array[i+1]
                array[i] = array[i+1], array[i]
                array[i]
                array[i]
                array[i] = array[i+1]
                array[i+1]
                array[i+1]
                array[i+1]
                array[i]
                array[i]
                array[i+1]
                array[i+1]
                array[i]
                array[i+1]
                array[i+1]
                array[i+1]
                print(cycleSort(array))

IndentationError: ignored

In [27]:
sample_input = "Write a python program that uses the Random Pivot Quick Sort to determine if any value appears in the array at least twice"
print('sample_input:', sample_input, '\n')
input_ids = tokenizer(sample_input, return_tensors='pt').input_ids
outputs = model.generate(input_ids, max_new_tokens=500)
print('generated output:\n', tokenizer.decode(outputs[0], skip_special_tokens=True))

sample_input: Write a python program that uses the Random Pivot Quick Sort to determine if any value appears in the array at least twice 

generated output:
 def quickSort(array):
    n = len(array)
    for i in range(n - 1):
        if array[i] > array[i+1]:
            array[i], array[i+1] = array[i+1], array[i+1]
            return array
arr = [2,3,4,5,6,7]
for i in range(len(arr)-1):
            if arr[i] > arr[i+1]:
                array[i] = arr[i+1], array[i]
print(arr[i] + arr[i+1] + arr[i+1] + arr[i+1] + arr[i+1] + arr[1] + arr[2] + arr[2] + arr[i+1]
print(arr[i])














In [30]:
def quickSort(array):
    n = len(array)
    for i in range(n - 1):
        if array[i] > array[i+1]:
            array[i], array[i+1] = array[i+1], array[i+1]
            return array
arr = [2,3,4,5,6,7]
for i in range(len(arr)-1):
            if arr[i] > arr[i+1]:
                array[i] = arr[i+1], array[i]
print(arr[i] + arr[i+1] + arr[i+1] + arr[i+1] + arr[i+1] + arr[1] + arr[2] + arr[2] + arr[i+1]
print(arr[i])

SyntaxError: ignored